In [ ]:
# Import packages
import TS_Custom
from TS_Custom import *


#initialize globalvariables
buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()


def getStrategyResult(symbol):
    #for symbol in symbol_list :
    try:
        #print (datetime.now().strftime('%H:%M'))         
        
        
        # get price history from tradestation
        min_2 = 2
        #min_5 = 5
        
        barsBack=int(barssince7am(min_2))
        if (barsBack <= 80):
            barsBack = 80
        
        
        df_2min = getPriceHistory_TS(symbol,str(barsBack), str(min_2))
        #df_5min = getPriceHistory_TS(symbol,str(barsBack), str(min_5))
        #df_2min = getPriceHistory_Polygon(symbol, str(min_2))
        
                    
        #convert current candles to heiken-ashi
        df_2min = getHA(df_2min)
        #df_5min = getHA(df_5min)

        # get options data from td ameritrade
        content = getOptionChainDatafromTD(symbol)

        # retrieve call symbol
        call_optionSymbol = getCallOptionSymbol(content)

        # retrieve put symbol
        put_optionSymbol = getPutOptionSymbol(content) 


        ema9 = df_2min['ema9']
        ema21 = df_2min['ema21']
        ema50 = df_2min['ema50']
        ema200 = df_2min['ema200']
        atr = df_2min['ATR']
        rsi = df_2min['RSI']
        stddev = df_2min['stddeviation']
        
       
        '''            
        if (symbol == "TSLA"):
            print(df_2min[['Close','Open','TimeStamp','RSI']])
            df_2min.to_csv(dfresults_filename, mode = 'w+', index=False)
        '''

        #retrieve keys
        last_key = df_2min['K_slow'].keys()[-1]
        previous_key = df_2min['K_slow'].keys()[-2]
        previous_key_2 = df_2min['K_slow'].keys()[-3]
        previous_key_3 = df_2min['K_slow'].keys()[-4]
        previous_key_4 = df_2min['K_slow'].keys()[-5]


        # Strategy Code

        #variables
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        strategy = 'BUY_EMA9_ANGLE'

        
        
        # Buy Stock  
        if (buylong == True):
            

            if ((df_2min['ema9_angle'][previous_key] >= 15) and \
                (df_2min['ema9_angle'][previous_key] > df_2min['ema9_angle'][previous_key_2]) and \
                (df_2min['linear_reg_slope'][previous_key] > df_2min['linear_reg_slope'][previous_key_2]) and \
                #(df_2min['linear_reg_slope'][previous_key] > 0) and \
                #(df_2min['HA_Close'][previous_key] > df_2min['HA_Open'][previous_key]) and \
                (df_2min['K_slow'][previous_key] > df_2min['D_slow'][previous_key]) and \
                (df_2min['K_slow'][previous_key] > df_2min['K_slow'][previous_key_2])and \
                (df_2min['K_slow'][previous_key] < 85) and \
                (rsi[previous_key] <= 80)) :
                
                '''
                if (((df_2min['linear_reg_slope'][previous_key] > 0) and \
                    (df_2min['linear_reg_slope'][previous_key_2] < 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] < 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_4] < 0))) : 
                '''
                
                
                if (not getSymbolStockPositionsfromDB(symbol)) :                        

                    # get quote from tradestation
                    quote = getQuote(symbol)
                    #midPrice = round((quote['Bid'] + quote['Ask'])/2, 2)
                    bidPrice = round((quote['Bid']), 2)
                    askPrice = round((quote['Ask']), 2)
                    #openPrice = round((df['Open'][last_key]), 2)
                    limitprice = askPrice
                    

                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)   

                    if (equity_bidaskpercent <= 0.4) :
                        
                        quantity=int(3800/(limitprice))

                        print("ALL CONDITIONS FOR BUY STOCK MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (bidPrice-limitprice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = mktvalue = round((quantity * bidPrice),2)
                        #triggerprice = round(quantity * atr[last_key],2)
                        triggerprice = round(quantity * 0.30 * stddev[last_key],2)
                        previous_openpl = round((quantity * (bidPrice-limitprice)), 2)
                        #profit_target = limitprice * 1.0065
                        profit_target = limitprice + float(triggerprice/quantity)
                        stop_loss = limitprice * 0.995

                        #buy Equity in TS
                        #buyEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)

                        # add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'BUY_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Long', quantity, limitprice,totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df_2min[['TimeStamp','Close','ema9','ema9_angle','linear_reg_slope','K_slow','D_slow']].tail(5))
                        #df_2min.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'BUY_STOCK', limitprice, quantity, '', bidPrice, askPrice,df_2min)


        '''
        elif (df['Close'][previous_key] < ema5[previous_key]) :
            positions = ts_client.getPositions(acct)
            for position in positions:                
                for key, val in position.items():                   
                    if (key.lower() == "symbol"):
                        val1 = val.split(" ")
                        if ((val1[0] == symbol) and val1[1].__contains__("C")):
                            sellCALL(symbol, quantity, df, last_key,  previous_key, ema5[previous_key], ema9[previous_key], val, tradelog_filename, content)    
        '''

        # Sell Short Stock   
        if (buyshort == True):

            if ((df_2min['ema9_angle'][previous_key] <= -15) and \
                (df_2min['ema9_angle'][previous_key] < df_2min['ema9_angle'][previous_key_2]) and \
                (df_2min['linear_reg_slope'][previous_key] < df_2min['linear_reg_slope'][previous_key_2]) and \
                #(df_2min['linear_reg_slope'][previous_key] < 0) and \
                #(df_2min['HA_Close'][previous_key] < df_2min['HA_Open'][previous_key]) and \
                (df_2min['K_slow'][previous_key] < df_2min['D_slow'][previous_key]) and \
                (df_2min['K_slow'][previous_key] < df_2min['K_slow'][previous_key_2]) and \
                (df_2min['K_slow'][previous_key] > 15) and \
                (rsi[previous_key] >= 25)) :
                
                '''
                if (((df_2min['linear_reg_slope'][previous_key] < 0) and \
                    (df_2min['linear_reg_slope'][previous_key_2] > 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] > 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_4] > 0))) :               
                '''                   
                
                if (not getSymbolStockPositionsfromDB(symbol)) :

                    # get quote from tradestation
                    quote = getQuote(symbol)
                    #midPrice = round((quote['Bid'] + quote['Ask'])/2, 2)
                    askPrice = round((quote['Ask']), 2)
                    bidPrice = round((quote['Bid']), 2)
                    #openPrice = round((df['Open'][last_key]), 2)
                    limitprice = bidPrice
                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)

                    if (equity_bidaskpercent <= 0.4) :

                        quantity=int(3800/(limitprice))

                        print("ALL CONDITIONS FOR SELLSHORT MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (limitprice-askPrice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = round((quantity * bidPrice),2)
                        
                        #triggerprice = round(quantity * atr[last_key],2)
                        triggerprice = round(quantity * 0.30 * stddev[last_key],2)
                        previous_openpl = round((quantity * (limitprice-askPrice)), 2)
                        #profit_target = limitprice * 0.935
                        profit_target = limitprice - float(triggerprice/quantity)
                        stop_loss = limitprice * 1.005

                        #sellshort Equity in TS
                        #sellshortEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)

                        #add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'SELLSHORT_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Short', quantity, limitprice, totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df_2min[['TimeStamp','Close','ema9','ema9_angle','linear_reg_slope','K_slow','D_slow']].tail(5))
                        #df_2min.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELLSHORT_STOCK', limitprice, quantity, '', bidPrice, askPrice,df_2min)


        '''
        elif (df['Close'][previous_key] > ema5[previous_key]):  
            positions = ts_client.getPositions(acct)
            for position in positions:                
                for key, val in position.items():                   
                    if (key.lower() == "symbol"):
                        val1 = val.split(" ")
                        if ((val1[0] == symbol) and val1[1].__contains__("P")):
                            sellPUT(symbol, quantity, df, last_key,  previous_key, ema5[previous_key], ema9[previous_key], val, tradelog_filename, content)    

        '''

        # Buy Options
        if (buycall == True):

            # Call Option
            if ((df_2min['ema9_angle'][previous_key] >= 25) and \
                (df_2min['ema9_angle'][previous_key] > df_2min['ema9_angle'][previous_key_2]) and \
                (df_2min['linear_reg_slope'][previous_key] > df_2min['linear_reg_slope'][previous_key_2]) and \
                (df_2min['HA_Close'][previous_key] > df_2min['HA_Open'][previous_key]) and \
                #(df_2min['HA_Close'][last_key] > df_2min['HA_Open'][last_key]) and \
                (rsi[previous_key] <= 70) and \
                (ema9[previous_key] > ema21[previous_key]) and \
                (ema21[previous_key] > ema50[previous_key])) :

                if (((df_2min['linear_reg_slope'][previous_key] > 0) and \
                    (df_2min['linear_reg_slope'][previous_key_2] < 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] > 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] < 0))) : 


                    #if (not getSymbolCallPositions(acct,symbol)) and (not getSymbolCallOrders_Received(acct,symbol)) :
                    if (not getSymbolCallPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "callExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "callExpDateMap")
                            bidPrice = getBidPrice(content, "callExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(3800/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY CALL OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Call Options in TS
                            #buyCALLStopLossProfitTarget(symbol, quantity, df, last_key, previous_key, ema5[previous_key], ema9[previous_key], call_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(call_optionSymbol), 'BUY_CALL', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(call_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(call_optionSymbol), strategy,'BUY_CALL', limitprice, quantity, '', bidPrice, askPrice,df_2min)



        # Put Option

        if (buyput == True):

            if ((df_2min['ema9_angle'][previous_key] < 0) and \
                (df_2min['linear_reg_slope'][previous_key] < df_2min['linear_reg_slope'][previous_key_2]) and \
                (df_2min['ema9_angle'][last_key] < 0) and \
                (df_2min['HA_Close'][previous_key] < df_2min['HA_Open'][previous_key]) and \
                #(df_2min['HA_Close'][last_key] < df_2min['HA_Open'][last_key]) and \
                (rsi[previous_key] >=35 ) and \
                (ema9[previous_key] < ema21[previous_key]) and \
                (ema21[previous_key] < ema50[previous_key])) :

                if (((df_2min['linear_reg_slope'][previous_key] < 0) and \
                    (df_2min['linear_reg_slope'][previous_key_2] > 0)) or \
                    ((df_2min['linear_reg_slope'][previous_key] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_2] < 0) and \
                     (df_2min['linear_reg_slope'][previous_key_3] > 0))) : 

                    #if (not getSymbolPutPositions(acct,symbol)) and (not getSymbolPutOrders_Received(acct,symbol)) :
                    if (not getSymbolPutPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "putExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "putExpDateMap")
                            bidPrice = getBidPrice(content, "putExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(3800/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY PUT OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Put Options in TS
                            #buyPUTStopLossProfitTarget(symbol, quantity, df, last_key,previous_key, ema5[previous_key], ema9[previous_key], put_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(put_optionSymbol), 'BUY_PUT', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(put_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(put_optionSymbol), strategy,'BUY_PUT', limitprice, quantity, '', bidPrice, askPrice,df_2min)


            #print("******************")

    except Exception as e:
        print(symbol)
        traceback.print_exc()
        print(e)
        
     
        
        
while True:        
        
    now = datetime.now()
    today800=now.replace(hour=7,minute=0,second=0,microsecond=0)
    today1500=now.replace(hour=15,minute=0,second=0,microsecond=0)
    if (now > today800 and now < today1500 ) :
        # import Multithreading file for this function below
        #symbol_list = getMomentumStocks()
        #print(symbol_list)
        #getStrategyResult(symbol_list)

        #get stocks list from multithread YF data
        #symbol_list = getStocks()
        #print('SYMBOL LIST', symbol_list)        

        buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()

        if (buystrategy == True) :   

                      
            symbol_list1 = pd.read_csv('tickers1.csv', header=None)[0].tolist()
            print(symbol_list1)
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list1)) as executor:
                executor.map(getStrategyResult,symbol_list1)

            # Total time calculated
            print(now, ' - symbol_list1 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)

            

            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list2 = pd.read_csv('tickers2.csv', header=None)[0].tolist()
            print(symbol_list2)
            
            
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list2)) as executor:            
                executor.map(getStrategyResult,symbol_list2)

            # Total time calculated
            print(now, ' - symbol_list2 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)



            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list3 = pd.read_csv('tickers3.csv', header=None)[0].tolist()
            print(symbol_list3)
            
            # Time starts from here
            start = time.time()        

            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list3)) as executor:
                #print(threading.currentThread().ident)
                executor.map(getStrategyResult,symbol_list3)

            # Total time calculated
            print(now, ' - symbol_list3 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)
            
            '''
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()  
            #symbol_list0=getMomentumStocks_TS_Scanner().head(10)['Symbol'].tolist()
            momentum_df=getVolumeMomentumStocks_TS_Scanner()
            if (not momentum_df.empty):
                symbol_list0=momentum_df.head(10)['Symbol'].tolist()
                print("Momentum Scanner Stocks:")
                print(symbol_list0)
                # Time starts from here
                start = time.time()
                if(len(symbol_list0)>0):
                    with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list0)) as executor:
                        executor.map(getStrategyResult,symbol_list0)

                # Total time calculated
                print(now, ' - symbol_list0 took ', time.time()-start, 'seconds.')

                print("**************************************************************")
                time.sleep(30)

            '''            
        
    
   


